In [1]:
import os
os.chdir("../../../")

import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel 
import spacy

In [2]:
target = os.getcwd() + "/data/text/png/post_courier_news.csv"
news = pd.read_csv(target).drop("Unnamed: 0", axis=1)

In [3]:
en = spacy.load('en_core_web_sm')
spacy_stopwords = list(en.Defaults.stop_words)

In [4]:
nltk_stopwords = stopwords.words("english")
print(len(nltk_stopwords))

179


In [5]:
nltk_unique = [sw for sw in nltk_stopwords if sw not in spacy_stopwords]
spacy_stopwords.extend(nltk_unique)

In [6]:
data = news.news.values.tolist()
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]
print(data[:1])

[' BY ANCILLA ILAVE A GROUP of students at the Legal Training Institute have been told that the most important part of their job is to get the facts right in court. Chief Justice Sir Gibbs Salika, who opened discussions at a week-long workshop in Port Moresby this week, told the students that getting facts right is important. “Know your facts well to present to the court, and actively participate in discussions,” Sir Gibbs said. He said this during the launch of the workshop on civil and criminal advocacy attended by LTI in collaboration with Australia’s Victorian Bar. The workshop was supported by the Australian Government through the PNG-Australia Partnership. The students were engaged in a range of activities, including interactive sessions and practical courtroom scenarios with accomplished trainers and legal practitioners. The experience provided participants with invaluable insights into case analysis, evidence presentation, effective communication, and persuasive techniques. Tho

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield (gensim.utils.simple_preprocess(str(sentence), deacc=True))

data_words = list(sent_to_words(data))
print(data_words[0])

['by', 'ancilla', 'ilave', 'group', 'of', 'students', 'at', 'the', 'legal', 'training', 'institute', 'have', 'been', 'told', 'that', 'the', 'most', 'important', 'part', 'of', 'their', 'job', 'is', 'to', 'get', 'the', 'facts', 'right', 'in', 'court', 'chief', 'justice', 'sir', 'gibbs', 'salika', 'who', 'opened', 'discussions', 'at', 'week', 'long', 'workshop', 'in', 'port', 'moresby', 'this', 'week', 'told', 'the', 'students', 'that', 'getting', 'facts', 'right', 'is', 'important', 'know', 'your', 'facts', 'well', 'to', 'present', 'to', 'the', 'court', 'and', 'actively', 'participate', 'in', 'discussions', 'sir', 'gibbs', 'said', 'he', 'said', 'this', 'during', 'the', 'launch', 'of', 'the', 'workshop', 'on', 'civil', 'and', 'criminal', 'advocacy', 'attended', 'by', 'lti', 'in', 'collaboration', 'with', 'australia', 'victorian', 'bar', 'the', 'workshop', 'was', 'supported', 'by', 'the', 'australian', 'government', 'through', 'the', 'png', 'australia', 'partnership', 'the', 'students', 'w

In [12]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[data_words], min_count=5, threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [13]:
def remove_stopwords(texts):
    return [[
        word for word in gensim.utils.simple_preprocess(str(doc))
        if word not in spacy_stopwords
    ] for doc in texts]


def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]


def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[0])

['group', 'student', 'legal', 'tell', 'important', 'job', 'fact', 'right', 'court', 'chief', 'open', 'discussion', 'week', 'long', 'week', 'tell', 'student', 'get', 'fact', 'right', 'important', 'know', 'fact', 'present', 'court', 'discussion', 'say', 'say', 'civil', 'criminal', 'advocacy', 'attend', 'support', 'australian', 'government', 'partnership', 'student', 'engage', 'range', 'activity', 'include', 'interactive', 'session', 'practical', 'courtroom', 'scenario', 'accomplish', 'trainer', 'legal', 'practitioner', 'experience', 'provide', 'participant', 'invaluable', 'insight', 'case', 'evidence', 'presentation', 'effective', 'communication', 'persuasive', 'technique', 'important', 'workshop', 'provide', 'mock', 'exercise', 'law', 'student', 'experience', 'actually', 'happen', 'court', 'room']


In [15]:
id2word = corpora.Dictionary(data_lemmatized)  
texts = data_lemmatized  
# DTM
corpus = [id2word.doc2bow(text) for text in texts]  

In [17]:
from gensim.models.wrappers import LdaMallet
mallet_path = os.getcwd() + '/../mallet-2.0.8/bin/mallet'
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

Mallet LDA: 20 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 1100
total tokens: 1462820
<10> LL/token: -10.2866
<20> LL/token: -9.36058
<30> LL/token: -8.78036
<40> LL/token: -8.54515

0	2.5	court law case order state process decision act issue land matter office rule lawyer justice legal application public petition chief 
1	2.5	public information country medium report include important book citizen region system protect national issue corruption write give publish communication note 
2	2.5	community people woman child support church youth life change live young violence family leader work program girl good bring member 
3	2.5	public city road water vehicle area travel proper clean resident place authority main town transport safety work carry highway drive 
4	2.5	event group visit year host culture local include hold support tourism promote company present traditional activity cultural day week sponsor 
5	2.5	people village affect area house fire goroka lose disaster

<160> LL/token: -8.18146
<170> LL/token: -8.17394
<180> LL/token: -8.17059
<190> LL/token: -8.16543

0	2.5	court law case process decision state order matter rule lawyer justice legal issue chief act application petition national file evidence 
1	2.5	system report information public medium provide citizen important agency country national set corruption include record book point practice understand conference 
2	2.5	community woman child church support youth life young change live family violence work program awareness girl group social society activity 
3	2.5	city water road public town vehicle place area travel clean resident transport safety main carry proper drive authority safe traffic 
4	2.5	event visit year host include hold culture group week promote tourism day support local attend present traditional cultural international meeting 
5	2.5	village area affect community assist fire assistance local people week team disaster house province island destroy appeal include early dama

<310> LL/token: -8.14909
<320> LL/token: -8.14724
<330> LL/token: -8.14598
<340> LL/token: -8.14655

0	2.5	court law case order state decision process matter rule issue lawyer justice legal national petition act office application file charge 
1	2.5	system report information public medium provide important citizen agency set corruption include national register process country require record state account 
2	2.5	woman community child church support youth life young violence live work family awareness program change group girl encourage society social 
3	2.5	city water public road town vehicle place area travel goroka clean resident transport proper main safe carry authority drive safety 
4	2.5	event visit host year culture include hold group promote day tourism present week local support traditional international stage cultural conference 
5	2.5	village area affect community week fire assist assistance team local disaster include house island appeal destroy province damage ground villa

<460> LL/token: -8.14396
<470> LL/token: -8.14559
<480> LL/token: -8.14434
<490> LL/token: -8.14404

0	2.5	court law case order state decision process matter issue rule lawyer justice legal petition national application act charge file evidence 
1	2.5	system report information public medium provide citizen include register process number agency set national important corruption record country base publish 
2	2.5	community woman child church support youth life young violence family awareness live change work group girl program society social man 
3	2.5	city public water road town vehicle place area market travel clean resident main transport goroka proper safety carry authority safe 
4	2.5	event visit year host culture day hold include group tourism promote week support present traditional international local stage cultural conference 
5	2.5	village area affect fire community local team week assistance province assist disaster water house include island destroy region month damage 
6	2.

<610> LL/token: -8.14474
<620> LL/token: -8.14554
<630> LL/token: -8.1435
<640> LL/token: -8.14499

0	2.5	court law case order state decision process matter rule lawyer justice legal national issue petition application file act evidence charge 
1	2.5	system report information public medium provide important country number register agency process set corruption citizen include record add ensure national 
2	2.5	community woman church support youth child life young violence awareness work change family group live girl society program social activity 
3	2.5	city public market road town place vehicle area water goroka travel clean resident proper main safety safe carry transport authority 
4	2.5	event visit year host day culture hold include tourism present group promote week local support traditional stage international cultural art 
5	2.5	village area affect water fire community assistance assist week team disaster local island region province include destroy house yesterday damage 
6	2.5

<760> LL/token: -8.14345
<770> LL/token: -8.14312
<780> LL/token: -8.14209
<790> LL/token: -8.14332

0	2.5	court law case order state decision process matter rule lawyer issue justice legal national petition application act file evidence charge 
1	2.5	system report information medium public provide agency important register number record national process corruption include base set place require citizen 
2	2.5	community woman church youth support child life young violence work awareness family live group change girl society social man activity 
3	2.5	city public market road town place area vehicle goroka main clean travel resident authority proper safe carry transport safety drive 
4	2.5	event visit year host day culture include tourism hold present promote group local stage traditional week international cultural support attend 
5	2.5	village area water affect fire community assist local assistance disaster team week island include destroy visit house damage appeal villager 
6	2.5	peo

<910> LL/token: -8.14192
<920> LL/token: -8.14142
<930> LL/token: -8.14164
<940> LL/token: -8.14167

0	2.5	court law case order state decision process rule matter lawyer justice legal issue petition charge application claim file national act 
1	2.5	system report information medium public provide process register important record number national agency corruption set ensure include require base publish 
2	2.5	community woman church support youth life young work violence awareness child live group change family society girl activity social man 
3	2.5	city public market town road area vehicle place travel goroka clean resident main proper safety safe transport ensure authority drive 
4	2.5	event visit year host day culture include hold tourism group local promote week traditional present international stage support cultural celebrate 
5	2.5	village water area affect fire community disaster assist week local assistance island team include month destroy house villager border damage 
6	2.5	p

In [18]:
def compute_coherence_values(dictionary,
                             corpus,
                             texts,
                             limit,
                             start=2,
                             step=1):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path,
                                                 corpus=corpus,
                                                 num_topics=num_topics,
                                                 id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model,
                                        texts=texts,
                                        dictionary=dictionary,
                                        coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [19]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=data_lemmatized,
                                                        start=2,
                                                        limit=20,
                                                        step=1)

Mallet LDA: 2 topics, 1 topic bits, 1 topic mask
Data loaded.
max tokens: 1100
total tokens: 1462820
<10> LL/token: -8.40038
<20> LL/token: -8.21858
<30> LL/token: -8.03796
<40> LL/token: -7.96905

0	25	people police election issue leader time officer public member provincial law day province report court yesterday week national order man 
1	25	government year school student work country support service project program community people district development health include provide child education provincial 

<50> LL/token: -7.93687
<60> LL/token: -7.91964
<70> LL/token: -7.90863
<80> LL/token: -7.90017
<90> LL/token: -7.89452

0	25	people police election issue leader time officer public member provincial day province law report court yesterday national family week order 
1	25	government year school country student work support service community project program district people development health include provide woman child education 

<100> LL/token: -7.89005
<110> LL/token: -7.88844
<12

<960> LL/token: -7.87017
<970> LL/token: -7.87006
<980> LL/token: -7.87061
<990> LL/token: -7.87024

0	25	people police election issue leader time officer public provincial member province law national report court yesterday day week order family 
1	25	government year school country student work support service community project program district woman development health people provide include child education 

<1000> LL/token: -7.87032

Total time: 42 seconds
Mallet LDA: 3 topics, 2 topic bits, 11 topic mask
Data loaded.
max tokens: 1100
total tokens: 1462820
<10> LL/token: -8.76873
<20> LL/token: -8.44997
<30> LL/token: -8.1906
<40> LL/token: -8.09005

0	16.66667	government school year student country program project support development education fund work include training district plan teacher high provide policy 
1	16.66667	people work service health community area road year child time family live local village province woman district life good day 
2	16.66667	police election issue 

<660> LL/token: -7.94256
<670> LL/token: -7.94251
<680> LL/token: -7.94303
<690> LL/token: -7.94456

0	16.66667	government school year student country support project program district service development work education fund include provide training plan high teacher 
1	16.66667	people woman health year community child area road time work family village live day life local good market church hospital 
2	16.66667	police people election issue leader provincial officer member public province government law report court national yesterday order case land candidate 

<700> LL/token: -7.94366
<710> LL/token: -7.94344
<720> LL/token: -7.9428
<730> LL/token: -7.94206
<740> LL/token: -7.94177

0	16.66667	government school year country student support project program district service development work education fund provide include training plan high provincial 
1	16.66667	people health woman year community child area road work time family village live day life local good market church hospital 
2

<310> LL/token: -7.99104
<320> LL/token: -7.98942
<330> LL/token: -7.98929
<340> LL/token: -7.99012

0	12.5	school government student project program service district year development support education work fund provide training plan health high provincial build 
1	12.5	people year country woman community work child good time leader support change life live church day visit family event bring 
2	12.5	police officer area village market hospital people man team operation health city water road week report vehicle accord month station 
3	12.5	government election issue provincial national law court people public member land candidate province process state leader office order governor pay 

<350> LL/token: -7.9908
<360> LL/token: -7.98908
<370> LL/token: -7.98905
<380> LL/token: -7.98943
<390> LL/token: -7.99036

0	12.5	school government student service project program year district development support education fund provide work training plan health high include build 
1	12.5	people year 

<910> LL/token: -7.99083
<920> LL/token: -7.99127
<930> LL/token: -7.99221
<940> LL/token: -7.99102

0	12.5	school government student service project program year district development support education fund provide work provincial health training plan high province 
1	12.5	people year country woman community work child good time support life change live church family visit day leader event bring 
2	12.5	police officer area people village market hospital man operation city road water team vehicle week report health accord month station 
3	12.5	government election issue people provincial national law court leader member public land candidate process state province office order governor pay 

<950> LL/token: -7.99212
<960> LL/token: -7.99148
<970> LL/token: -7.99113
<980> LL/token: -7.99007
<990> LL/token: -7.99101

0	12.5	government school student service project program year district development education support fund work provide provincial training plan health high include 
1	12.5	peo

...

<310> LL/token: -8.16777
<320> LL/token: -8.1688
<330> LL/token: -8.16925
<340> LL/token: -8.16446

0	2.63158	program training work officer train skill staff week provide management participant service manager conduct knowledge job base team director experience 
1	2.63158	system plan policy improve increase level national sector provide ensure agency base implement service management high department address include develop 
2	2.63158	country development government economic region economy opportunity partner support partnership world papua important meeting investment policy develop trade resource international 
3	2.63158	year child church life time family live work good young mother give serve today thing hard start age day hope 
4	2.63158	election candidate vote open polling official national electorate return count seat team period process province general yesterday campaign day conduct 
5	2.63158	police officer charge station man report arrest law order suspect investigation securit

<460> LL/token: -8.16096
<470> LL/token: -8.1625
<480> LL/token: -8.16137
<490> LL/token: -8.16229

0	2.63158	program training work officer train skill staff provide week management base participant conduct team manager director knowledge job experience attend 
1	2.63158	system plan policy improve national ensure level increase sector provide agency service implement base management public address department develop high 
2	2.63158	country development economic region economy support opportunity partner world government papua important investment partnership trade international future develop meeting business 
3	2.63158	year child church life time family live work mother young good give serve today day thing age hard word start 
4	2.63158	election candidate vote open polling national official electorate return team count seat process period week general yesterday conduct province campaign 
5	2.63158	police officer charge report station man arrest suspect vehicle investigation law order 

<610> LL/token: -8.15916
<620> LL/token: -8.15743
<630> LL/token: -8.15809
<640> LL/token: -8.15818

0	2.63158	program training work officer provide train staff skill week management participant team manager knowledge director conduct job experience base service 
1	2.63158	plan system policy national improve level ensure sector increase provide agency implement base service public management government include department address 
2	2.63158	country development economic region support economy opportunity partner world papua develop government partnership meeting include future investment international important trade 
3	2.63158	year child church time life family live work good serve mother young give day thing today bring start hard word 
4	2.63158	election candidate vote open polling official national electorate return count seat team process yesterday period conduct general week campaign province 
5	2.63158	police officer man station charge report arrest law order suspect vehicle secur

<760> LL/token: -8.16487
<770> LL/token: -8.16487
<780> LL/token: -8.16486
<790> LL/token: -8.16494

0	2.63158	program training work officer provide train skill staff week management participant conduct job manager knowledge attend experience director team base 
1	2.63158	system plan policy national ensure improve level provide service public sector agency increase implement base address information department management develop 
2	2.63158	country development economic region economy support world partner opportunity papua partnership government international meeting investment future trade develop important resource 
3	2.63158	year child church time family life live work mother good serve young give day today start hard thing word hope 
4	2.63158	election candidate vote open polling official national electorate team return count process seat yesterday period general conduct week campaign start 
5	2.63158	police officer man station report charge arrest vehicle suspect law security inves

<910> LL/token: -8.16565
<920> LL/token: -8.16725
<930> LL/token: -8.16731
<940> LL/token: -8.16562

0	2.63158	program training work officer provide train skill staff week management participant job manager conduct knowledge experience director team opportunity base 
1	2.63158	system plan policy national ensure service improve provide level public sector agency implement department information base management include increase address 
2	2.63158	country development economic region support economy partner world papua partnership opportunity include meeting future investment develop international trade business important 
3	2.63158	year child church family time life live work mother good young serve give day today thing start bring hard word 
4	2.63158	election candidate vote polling open national official electorate return team count seat process general period yesterday week conduct campaign today 
5	2.63158	police officer man station report arrest vehicle charge suspect order security 

In [21]:
best_model = model_list[np.argmax(coherence_values)]
best_model.print_topics()

[(0,
  '0.034*"business" + 0.028*"market" + 0.027*"company" + 0.016*"local" + 0.014*"money" + 0.014*"sell" + 0.012*"food" + 0.011*"agriculture" + 0.011*"good" + 0.011*"small"'),
 (1,
  '0.053*"woman" + 0.053*"community" + 0.034*"people" + 0.024*"church" + 0.023*"work" + 0.023*"support" + 0.022*"change" + 0.019*"youth" + 0.019*"leader" + 0.017*"program"'),
 (2,
  '0.032*"court" + 0.021*"land" + 0.018*"state" + 0.016*"issue" + 0.016*"pay" + 0.016*"landowner" + 0.015*"process" + 0.014*"order" + 0.014*"decision" + 0.014*"law"'),
 (3,
  '0.046*"road" + 0.046*"work" + 0.025*"build" + 0.017*"complete" + 0.016*"building" + 0.015*"travel" + 0.014*"open" + 0.014*"cost" + 0.011*"transport" + 0.011*"start"'),
 (4,
  '0.076*"government" + 0.061*"project" + 0.059*"district" + 0.042*"service" + 0.038*"fund" + 0.035*"year" + 0.028*"support" + 0.027*"people" + 0.024*"development" + 0.023*"program"'),
 (5,
  '0.044*"year" + 0.035*"family" + 0.032*"time" + 0.028*"child" + 0.017*"life" + 0.014*"day" + 0.0

In [26]:
topics_df = pd.DataFrame()
for idx, val in enumerate(best_model.load_document_topics()):
    temp_df = pd.DataFrame(val)
    if topics_df.empty:
        topics_df = temp_df
    else:
        topics_df = pd.concat([topics_df, temp_df], axis=1)

In [28]:
topics_df_new = topics_df.drop(0, axis=1).T.reset_index().drop("index", axis=1)
topics_df_new["dominant_topic"] = [np.argmax(row) for _, row in topics_df_new.iterrows()]
inclu_indexs = topics_df_new[topics_df_new.dominant_topic.isin([0, 13])].index.to_list()

In [29]:
busecon_news = news.iloc[inclu_indexs].reset_index().drop("index", axis=1)
busecon_news

,url,title,date,news,tag
0,https://www.postcourier.com.pg/fresh-vegetable...,Fresh vegetables galore at Mt Hagen market,"July 13, 2023",\nBY PEARSON KOLO\nMT Hagen market in the West...,No Tag
1,https://www.postcourier.com.pg/cocoa-freight-p...,Cocoa freight pact helps rural growers,"July 13, 2023",\nBy PAUL BUNGTABU\nThe PNG Cocoa Board rural ...,No Tag
2,https://www.postcourier.com.pg/french-celebrat...,French celebrate Bastille Day in POM,"July 19, 2023",\nIn a remarkable celebration held at the APEC...,"Tagged: Bastille Day, French Embassy in Papua ..."
3,https://www.postcourier.com.pg/western-highlan...,Western Highlands is top SME province,"July 18, 2023",\nBY PEARSON KOLO\nWESTERN Highlands Province ...,No Tag
4,https://www.postcourier.com.pg/enb-takes-on-ic...,ENB takes on ICT revolution,"July 20, 2023",\nTHE digital transformation workshop that is ...,No Tag
...,...,...,...,...,...
1991,https://www.postcourier.com.pg/govt-gives-usai...,Govt gives USAID backing,"December 17, 2015",\nTHE United States Agency for International D...,No Tag
1992,https://www.postcourier.com.pg/santa-comes-ear...,Santa comes early for single Mt Hagen mum genset,"December 17, 2015",\nBy RAMCY WAMA\nA ROADSIDE bottle collecting ...,No Tag
1993,https://www.postcourier.com.pg/plan-set-to-add...,Plan set to address effects of El Nino,"December 17, 2015",\nA MAJOR review of strategies and operational...,No Tag
1994,https://www.postcourier.com.pg/housing-plan-se...,Housing plan set to go,"December 16, 2015",\nKey stakeholders sign MOU\nBy JOHN PANGKATAN...,No Tag


In [30]:
# ss_df = pd.DataFrame()
# for news in text_lst[:-1]:
#     news_lst = news.split("All rights reserved.")
#     header = news_lst[0]
#     header_lst = header.split("\n")
#     topic = header_lst[0]
#     sample = news_lst[-1].replace("\n", " ").strip()
    
#     sid = SentimentIntensityAnalyzer()
#     ss = sid.polarity_scores(sample)
#     ss_temp = pd.DataFrame(ss, columns=ss.keys(), index=[0])
#     date_pattern = r'\b\d{1,2} (?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4}\b'
#     ss_temp["date"] = re.findall(date_pattern, c[0])[0]
#     ss_temp["text"] = sample
#     ss_df = pd.concat([ss_df, ss_temp], axis=0)

ss_df = pd.DataFrame()
for news in busecon_news.news:
    sample = news.replace("\n", " ").strip()
    
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sample)
    ss_temp = pd.DataFrame(ss, columns=ss.keys(), index=[0])
    ss_df = pd.concat([ss_df, ss_temp], axis=0)

In [31]:
busecon_news["sentiment"] = ss_df["compound"].tolist()
busecon_news["date"] = pd.to_datetime(busecon_news["date"])

In [53]:
sent_by_month = (busecon_news.set_index("date").groupby(pd.Grouper(freq="m"))[["sentiment"]].mean().reset_index())
sent_by_month['date'] = (sent_by_month['date'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))

In [54]:
png_infl_path = os.getcwd() + "/data/prices/outputs/2017/RTFP_country_2017_2022-08-22.csv"
png_infl = pd.read_csv(png_infl_path)
png_infl["date"] = pd.to_datetime(png_infl["date"], dayfirst=True)
png_infl_sent = png_infl.merge(sent_by_month, how="left", on="date")
png_infl_sent.head(5)

,Open,High,Low,Close,Inflation,country,ISO3,date,sentiment
0,0.997417,1.035504,0.960535,0.997725,NaN,Papua New Guinea,PNG,2017-01-01,0.617313
1,0.998328,1.037044,0.960832,1.037044,NaN,Papua New Guinea,PNG,2017-02-01,0.861838
2,1.029174,1.066052,0.992296,1.012038,NaN,Papua New Guinea,PNG,2017-03-01,0.697121
3,1.019978,1.059257,0.980700,1.028941,NaN,Papua New Guinea,PNG,2017-04-01,0.889195
4,1.018736,1.061383,0.976089,1.009300,NaN,Papua New Guinea,PNG,2017-05-01,0.717284


In [90]:
sent_change = []
for i in range(len(png_infl_sent)):
    if i < 12:
        sent_change.append(np.NaN)
    else:
        a = (png_infl_sent["sentiment"][i] - png_infl_sent["sentiment"][i-12])/png_infl_sent["sentiment"][i-12]
        sent_change.append(a)

In [93]:
png_infl_sent["sent_change"] = sent_change

In [113]:
png_infl_sent["infl_diff"] = png_infl_sent["Close"].diff()
png_infl_sent["sent_diff"] = png_infl_sent["sentiment"].diff()

In [112]:
from scipy import signal
correlation = signal.correlate(c["sent_change"], c["Inflation"], mode="full")
lags = signal.correlation_lags(len(c["sent_change"]), len(c["Inflation"]), mode="full")
lag = lags[np.argmax(correlation)]
lag

27

In [118]:
import statsmodels.formula.api as smf
mod = smf.ols("infl_diff~sent_diff", data=png_infl_sent)
res = mod.fit(cov_type='HAC',cov_kwds={'maxlags':12})
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              infl_diff   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.028
Date:                Mon, 21 Aug 2023   Prob (F-statistic):              0.314
Time:                        17:56:57   Log-Likelihood:                 108.42
No. Observations:                  67   AIC:                            -212.8
Df Residuals:                      65   BIC:                            -208.4
Df Model:                           1                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0048      0.003      1.422      0.155      -0.002       0.011
sent_diff     -0.0225      0.022     -1.014      0.311      -0.066       0.021
==============================================================================
Omnibus:                        0.098   Durbin-Watson:                   2.491
Prob(Omnibus):                  0.952   Jarque-Bera (JB):                0.062
Skew:                          -0.062   Prob(JB):                        0.969
Kurtosis:                       2.917   Cond. No.                         3.69
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 12 lags and without small sample correction
"""